# Chapter 2

Example project end to end.

# TOC

1. [Accessing Real Data](#real_data)
2. [Used Data](#used_data)
3. Project
   1. Loog at the big picture
   2. Get the data.
   3. Discover and visualize the data to gain insights.
   4. Prepare the data for Machine Learning algorithms.
   5. Select a model and train it.
   6. Fine-tune your model.
   7. Present your solution.
   8. Launch, monitor, and maintain your system.

<a id='real_data'></a>
## Accessing Real Data

**Popular open data repositories**

* [UC Irvine Machine Learning Repository](http://archive.ics.uci.edu/ml/)
* [Kaggle datasets](https://www.kaggle.com/datasets)
* [Amazon’s AWS datasets](http://aws.amazon.com/fr/datasets/)

**Meta portals** (they list open data repositories)

* http://dataportals.org/
* http://opendatamonitor.eu/
* http://quandl.com/

**Other pages listing many popular open data repositories**

* [Wikipedia’s list of Machine Learning datasets](https://goo.gl/SJHN2k)
* [Quora.com question](http://goo.gl/zDR78y)
* [Datasets subreddit](https://www.reddit.com/r/datasets)

<a id='used_data'></a>
## Used Data

California Housing Prices dataset from the StatLib repository<br />
<img src="images/C02_california_real_state_dataset.png" style="width: 500px" />